## Bienvenido a la segunda práctica de Jupyter Notebook

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/jpcano1/MINE-4101-tareas/blob/master/lab_2.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png"/>Run in Google Colab</a>
  </td>
</table>

In [9]:
!wget -q "https://github.com/jpcano1/MINE-4101-tareas/blob/master/datasets/CAPITULO%20B.%20CONDICIONES%20DE%20LA%20VIVIENDA.txt?raw=true" -O "condiciones_vivienda.txt"
!wget -q "https://github.com/jpcano1/MINE-4101-tareas/blob/master/datasets/CAPITULO%20H.%20FUERZA%20DE%20TRABAJO%20-%20Uso%20del%20tiempo%20e%20ingresos.txt?raw=true" -O "fuerza_trabajo.txt"

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use("ggplot")
import seaborn as sns

In [19]:
data_condiciones = pd.read_table("condiciones_vivienda.txt")
data_fuerza = pd.read_table("fuerza_trabajo.txt")

In [20]:
columns_condiciones = {
    "FEX_C": "factores_expansion",
    "P424": "tipo_vivienda",
    "P4030S1": "energia_electrica",
    "P4030S1A1": "estrato",
    "P4030S5": "acueducto",
    "P4030S3": "alcantarillado",
    "P4030S4": "recoleccion_basuras",
    "P4030S4A1": "dias_x_semana",
    "P4030S2": "gas_natural_dom",
    "P4020": "material_pisos",
    "DIA_REFERENCIA_2": "dia_referencia"
}

In [21]:
data_condiciones.rename(columns=columns_condiciones, inplace=True)
data_condiciones.rename(columns=lambda x: x.lower(), inplace=True)

In [49]:
columns_fuerza_list = ["oficio_princ", "rama_princ", 
                       "lugar_trabajo", "contrato", "misma_empresa", 
                       "horas_semana", "trabajo_dia", "horas_trabajo", 
                       "minutos_trabajo",  "total_ida", "horas_ida", 
                       "minutos_ida",  "motivo_no_trabajo", "este_trabajo", 
                       "ganancias_mes_pasado", "ganancia_neta_mes_pasado", 
                       "numero_meses", "cantidad_tiempo", "numero_personas",
                       "empleo_secundario", "horas_semana_empleo_secundario", 
                       "trabajo_secundario_dia", "horas_trabajo_secundario", 
                       "minutos_trabajo_secundario", "ida_secundario_horas", 
                       "ida_secundario_minutos",  "ganancia_secundaria_mes_pasado", 
                       "ganancia_jubilacion", "valor_jubilacion", "ganancia_arriendo",  
                       "valor_arriendo",  "ganancia_ayudas", "ayuda_provino_de", 
                       "valor_ayudas"]

In [52]:
index = 0
columns_fuerza = {}
for col in data_fuerza.columns[3:]:
    columns_fuerza[col] = columns_fuerza_list[index]
    index += 1

In [ ]:
columns_fuerza

In [ ]:
data_fuerza.rename(columns=columns_fuerza)

In [ ]:
data_fuerza.rename(columns=lambda x: x.lower())